In [1]:
# cd /kaggle/working

In [2]:
# !git clone https://github.com/naver/r2d2

In [3]:
# cd r2d2


In [9]:
import os, pdb
import torch
import torch.optim as optim

from tools import common, trainer
from tools.dataloader import *
from nets.patchnet import *
from nets.losses import *
from PIL import Image, ImageOps
from tqdm.notebook import tqdm 


In [10]:
# try:
#     DataPreparation
# except NameError:
#     os.system("cp -r /kaggle/input/naturalimages/ /tmp/dataset")
#     os.system("ln -s /tmp/dataset /kaggle/working/r2d2/data")
#     DataPreparation = True
# else:
#     pass

In [11]:
# !cp -r /kaggle/input/naturalimages/ /tmp/dataset
!unlink data
!ln -s /run/media/javid/App/DataSet/r2d2/ data

In [12]:
# ls /tmp/dataset

In [13]:
# ls

In [14]:
from  datasets import *

Dataset SAR_db_images not available, reason: Empty SAR dataset


In [15]:
class StillTransform (object):
    """ Takes and return an image, without changing its shape or geometry.
    """
    def _transform(self, img):
        raise NotImplementedError()
        
    def __call__(self, inp):
        img = F.grab_img(inp)

        # transform the image (size should not change)
        try:
            img = self._transform(img)
        except TypeError:
            pass

        return F.update_img_and_labels(inp, img, persp=(1,0,0,0,1,0,0,0))

    
class PixelSpeckleNoise (StillTransform):
    """ Takes an image, and add random white noise.
    """
    def __init__(self, var=.05, seed=None):
        StillTransform.__init__(self)
        assert 0 <= var < 1
        self.var = var
        self.seed = seed

    def __repr__(self):
        return "PixelSpeckleNoise(%g)" % self.var
    
    def normalize(self, img,minimum=0, maximum=1):
        img_max = np.max(img)
        img_min = np.min(img)
        return (img-img_min)/np.abs(img_max-img_min)*(maximum-minimum)+minimum

    def _transform(self, img):
        normalized_img = self.normalize(img)
        upper_band = (12*self.var)**.5
        np.random.seed(self.seed)
        noise = np.random.uniform(-upper_band/2,upper_band/2,size=img.shape)
        noisy_img = normalized_img*(1+noise)
        noisy_img = np.clip(noisy_img,0,1)
        ret_val = self.normalize(noisy_img,maximum=255)
        return  Image.fromarray(np.uint8(ret_val))
    
class PixelNoise (StillTransform):
    """ Takes an image, and add random white noise.
    """
    def __init__(self, ampl=20):
        StillTransform.__init__(self)
        assert 0 <= ampl < 255
        self.ampl = ampl

    def __repr__(self):
        return "PixelNoise(%g)" % self.ampl

    def _transform(self, img):
        img = np.float32(img)
        img += np.random.uniform(0.5-self.ampl/2, 0.5+self.ampl/2, size=img.shape)
        return Image.fromarray(np.uint8(img.clip(0,255)))


In [16]:
# default_net = "Quad_L2Net_ConfCFS()"
default_net = "Custom_Quad_L2Net_ConfCFS()"


toy_db_debug = """SyntheticPairDataset(
    ImgFolder('imgs'), 
            'RandomScale(256,1024,can_upscale=True)', 
            'RandomTilting(0.5), PixelSpeckleNoise(.5)')"""

db_web_images = """SyntheticPairDataset(
    web_images, 
        'RandomScale(256,1024,can_upscale=True)',
        'RandomTilting(0.5), PixelSpeckleNoise(.5)')"""

db_aachen_images = """SyntheticPairDataset(
    aachen_db_images, 
        'RandomScale(256,1024,can_upscale=True)', 
        'RandomTilting(0.5), PixelSpeckleNoise(.5)')"""

db_aachen_style_transfer = """TransformedPairs(
    aachen_style_transfer_pairs,
            'RandomScale(256,1024,can_upscale=True), RandomTilting(0.5), PixelSpeckleNoise(.5)')"""

db_aachen_flow = "aachen_flow_pairs"


db_sar_images = """SyntheticPairDataset(
    sar_db_images, 
        'RandomScale(256,1024,can_upscale=True)', 
        'RandomTilting(0.5), PixelSpeckleNoise(.5)')"""



In [17]:

default_dataloader = """PairLoader(CatPairDataset(`data`),
    scale   = 'RandomScale(256,1024,can_upscale=True)',
    distort = 'ColorJitter(0.2,0.2,0.2,0.1)',
    crop    = 'RandomCrop(192)')"""

# default_sampler = """NghSampler2(ngh=7, subq=-8, subd=1, pos_d=3, neg_d=5, border=16,
#                             subd_neg=-8,maxpool_pos=True)"""

default_sampler = """DoubleDescNghSampler2(ngh=7, subq=-8, subd=1, pos_d=3, neg_d=5, border=16,
                            subd_neg=-8,maxpool_pos=True, desc_dim=128)"""




default_loss = """MultiLoss(
        1, ReliabilityLoss(`sampler`, base=0.5, nq=20),
        1, CosimLoss(N=`N`),
        1, PeakyLoss(N=`N`))"""


In [18]:
data_sources = dict(
#     D = toy_db_debug,
#     W = db_web_images,
    A = db_aachen_images,
#     F = db_aachen_flow,
#     S = db_aachen_style_transfer,
#     X = db_sar_images
    )


In [19]:
class MyTrainer(trainer.Trainer):
    """ This class implements the network training.
        Below is the function I need to overload to explain how to do the backprop.
    """
    def forward_backward(self, inputs):
        output = self.net(imgs=[inputs.pop('img1'),inputs.pop('img2')])
        allvars = dict(inputs, **output)
        loss, details = self.loss_func(**allvars)
        if torch.is_grad_enabled(): loss.backward()
        return loss, details



In [20]:

def load_network(model_fn): 
    checkpoint = torch.load(model_fn)
    print("\n>> Creating net = " + checkpoint['net']) 
    net = eval(checkpoint['net'])
    nb_of_weights = common.model_size(net)
    print(f" ( Model size: {nb_of_weights/1000:.0f}K parameters )")

    # initialization
    weights = checkpoint['state_dict']
    net.load_state_dict({k.replace('module.',''):v for k,v in weights.items()})
    return net.eval()


In [21]:
mkdir trained_models

In [26]:
save_path = "./trained_models"
gpu = 0
train_data = "A"
data_loader = default_dataloader
threads = 8
batch_size = 8
net = default_net
sampler = default_sampler
N = patch_size = 16 
loss = default_loss
learning_rate = 1e-4
weight_decay = 5e-4
epochs = 10
network_path = "./models/faster2d2_WASF_N16.pt"

In [27]:
iscuda = common.torch_set_gpu(gpu)
common.mkdir_for(save_path)


Launching on GPUs 0


In [28]:
db = [data_sources[key] for key in train_data]


In [29]:
data_loader.replace('`data`',','.join(db)).replace('\n','')

"PairLoader(CatPairDataset(SyntheticPairDataset(    aachen_db_images,         'RandomScale(256,1024,can_upscale=True)',         'RandomTilting(0.5), PixelSpeckleNoise(.5)')),    scale   = 'RandomScale(256,1024,can_upscale=True)',    distort = 'ColorJitter(0.2,0.2,0.2,0.1)',    crop    = 'RandomCrop(192)')"

In [30]:
# Create data loader
db = [data_sources[key] for key in train_data]
db = eval(data_loader.replace('`data`',','.join(db)).replace('\n',''))
print("Training image database =", db)
loader = threaded_loader(db, False, threads, batch_size, shuffle=True)


Training image database = PairLoader
CatPairDataset(Dataset: SyntheticPairDataset   4479 images and pairs   root: data/aachen...   Scale: Compose(    RandomScale(0))   Distort: Compose(    RandomTilt(0.5, 'all')    PixelSpeckleNoise(0.5)) )  npairs: 4479
  Distort: ColorJitter(0.2,0.2,0.2,0.1), 
  Crop: RandomCrop((192, 192)), 
  Norm: ToTensor(),  Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 



In [31]:
for a in loader:
    print(a)
    break
    

/home/javid/.conda/envs/tensorflow/lib/python3.8/site-packages/torchvision/transforms/functional.py:133: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()
/home/javid/.conda/envs/tensorflow/lib/python3.8/site-packages/torchvision/transforms/functional.py:133: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be supp

{'img1': tensor([[[[ 1.1700,  1.1015,  1.1700,  ...,  1.1872,  0.9817,  0.7248],
          [ 0.9303,  0.9988,  1.3584,  ...,  0.5707,  0.5878,  0.5364],
          [ 0.7591,  0.8789,  0.9646,  ...,  0.6392,  0.6734,  0.6221],
          ...,
          [-1.4672, -1.3815, -1.3473,  ..., -1.2617, -1.1932, -1.2788],
          [-1.3473, -1.2959, -1.4158,  ..., -1.2445, -1.1932, -1.2788],
          [-1.4843, -1.4843, -1.3987,  ..., -1.3644, -1.2445, -1.2788]],

         [[ 1.0630,  0.9930,  1.0630,  ...,  1.0630,  0.8354,  0.5903],
          [ 0.8354,  0.9230,  1.2556,  ...,  0.4153,  0.4328,  0.3627],
          [ 0.6429,  0.7479,  0.8354,  ...,  0.5378,  0.5203,  0.4503],
          ...,
          [-1.1429, -1.0553, -1.0378,  ..., -0.9853, -0.9328, -1.0203],
          [-1.0203, -0.9853, -1.1078,  ..., -0.9853, -0.8978, -0.9853],
          [-1.1604, -1.1779, -1.0903,  ..., -1.1078, -0.9328, -0.9328]],

         [[ 0.8971,  0.8274,  0.8971,  ...,  0.8448,  0.7054,  0.5311],
          [ 0.6879,  

/run/media/javid/Entertainment/arco_home/ml/projects/p30_thesis/r2d2_sar/tools/dataloader.py:302: UserWarning: An output with one or more elements was resized since it had shape [884736], which does not match the required output shape [8, 3, 192, 192].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:24.)
  return torch.stack(batch, 0, out=out)
/run/media/javid/Entertainment/arco_home/ml/projects/p30_thesis/r2d2_sar/tools/dataloader.py:302: UserWarning: An output with one or more elements was resized since it had shape [884736], which does not match the required output shape [8, 3, 192, 192].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by re

In [32]:
b

NameError: name 'b' is not defined

In [31]:
g

In [28]:
next(g)

TypeError: 'NoneType' object is not an iterator

In [34]:
# net = load_network(network_path)

net = Custom_Quad_L2Net_ConfCFS()

In [35]:
net

Custom_Quad_L2Net_ConfCFS(
  (ops): ModuleList(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, tr

In [ ]:
# # initialization
# pretrained = "./models/faster2d2_WASF_N16.pt"
# checkpoint = torch.load(pretrained, lambda a,b:a)
# net.load_pretrained(checkpoint['state_dict'])


In [36]:
# create losses
loss = loss.replace('`sampler`',sampler).replace('`N`',str(patch_size))
print("\n>> Creating loss = " + loss)
loss = eval(loss.replace('\n',''))



>> Creating loss = MultiLoss(
        1, ReliabilityLoss(NghSampler2(ngh=7, subq=-8, subd=1, pos_d=3, neg_d=5, border=16,
                            subd_neg=-8,maxpool_pos=True), base=0.5, nq=20),
        1, CosimLoss(N=16),
        1, PeakyLoss(N=16))


In [37]:
# create optimizer
optimizer = optim.Adam( [p for p in net.parameters() if p.requires_grad], 
                        lr=learning_rate, weight_decay=weight_decay)

train = MyTrainer(net, loader, loss, optimizer)
if iscuda: train = train.cuda()


In [ ]:
# Training loop #
for epoch in range(epochs):
    print(f"\n>> Starting epoch {epoch}...")
    train()

print(f"\n>> Saving model to {save_path}")

NameError: name 'args' is not defined

'./trained_models/saved_model.pt'

In [46]:
torch.save({'net': "Double_descriptor", 'state_dict': net.state_dict()}, save_path+"/saved_model.pt")